# Liver Model Construction: Notebook

In [1]:
import os
import re
import warnings
from cobra.io import *
from cobra.io.json import *
import escher
import mass
import numpy as np
import pandas as pd
import equilibrator_api
from equilibrator_api import ComponentContribution, Q_
import sympy as sym
from cobra import Model, Reaction, Metabolite
import cobra.test
from os.path import join
from mass.util import qcqa
from mass.util import qcqa_model
from cobra import DictList
from mass import (
    MassConfiguration, MassMetabolite, MassModel,
    MassReaction, Simulation, UnitDefinition)
from mass.io.json import save_json_model as save_json_mass_model
from mass.visualization import plot_comparison, plot_time_profile
from mass.visualization import (
    plot_ensemble_phase_portrait, plot_ensemble_time_profile)
mass_config = MassConfiguration()
mass_config.irreversible_Keq = float("inf")
print("MASSpy version: {0}".format(mass.__version__))
from six import iteritems
import matplotlib.pyplot as plt
from mass.thermo import (
    ConcSolver, sample_concentrations,
    update_model_with_concentration_solution)
from cobra.io.json import *
import cobra_dict as c_d
import csv
import altair as alt
from minspan.minspan import minspan,nnz
# from minspan import minspan,nnz

MASSpy version: 0.1.6


In [2]:
import sys
# from minspan import minspan,nnz
print(sys.version)

3.8.12 (default, Oct 12 2021, 03:01:40) [MSC v.1916 64 bit (AMD64)]


In [3]:
maps_dir = os.path.abspath("maps")
data_dir = os.path.abspath("data")
dir = os.path.abspath("")
model_dir = os.path.abspath("models")
minspan_dir= os.path.abspath("minspans_csv")

### Getting reactions and metabolite data from RECON 3D

In [4]:
# making dataframe of metabolites

csv_met = os.path.join(data_dir,"glycolysis_gluconeogenesis_met_df.csv")
met_csv_df = pd.read_csv(csv_met,index_col="id")
metabolite_info=met_csv_df.to_dict(orient='index')

# making dataframe of reactions
csv_rxn = os.path.join(data_dir,"glycolysis_gluconeogenesis_reaction_df.csv")
rxn_csv_df = pd.read_csv(csv_rxn,index_col="id") 
reaction_info = rxn_csv_df.to_dict(orient='index')

### Loading cobra model

In [5]:
# Initiate empty model
trial= cobra.Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2022-03-01


In [6]:
#  Add all the remaining metabolites involved in the pathway
for met_id, info in metabolite_info.items():
    met = cobra.Metabolite(met_id, name=info['name'], formula=info['formula'], 
                           charge=info['charge'], compartment=info['compartment'])
    trial.add_metabolites(met)

In [7]:
# Loading reaction data as JSON file to maintain data types as dictionaries 
rxn_json = os.path.join(data_dir,"glycolysis_gluconeogenesis_reaction_df.json")
with open(rxn_json) as json_file:
    rxn_data = json.load(json_file)

In [8]:
for reaction, info in rxn_data.items():
    reaction_obj = cobra.Reaction(reaction)
    reaction_obj.id=reaction
    reaction= reaction_obj.id
    reaction_obj.lower_bound = info['lower_bound']
    reaction_obj.upper_bound = info['upper_bound']
    reaction_obj.name = info['name']
    trial.add_reaction(reaction_obj)
    temp=info['metabolites']
    reaction_obj.add_metabolites(temp)
    print(reaction)

HEX1
PGI
FBP
PFK
FBA
TPI
GAPD
PGK
PGM
ENO
PEPCKm
PYK
PCm
LDH_L
PYRt2m
ADK1
ATPM


In [9]:
#adding NADHM 
nadhm = {'nadh_c': -1, 'h_c': 1, 'nad_c': 1}
rxn = cobra.Reaction('NADHM')
trial.add_reaction(rxn)
rxn.add_metabolites(nadhm)




peptm = {'pep_m': -1, 'pi_c': -1, 'pi_m': 1, 'pep_c':1}
rxn2 = cobra.Reaction('PEPtm')
trial.add_reaction(rxn2)
rxn2.add_metabolites(peptm)



g6pase = {'g6p_c': -1, 'h2o_c': -1, 'pi_c': 1, 'glc__D_c':1}
rxn3 = cobra.Reaction('G6PP')
trial.add_reaction(rxn3)
rxn3.add_metabolites(g6pase)
trial.reactions.G6PP.lower_bound=0



In [10]:
#Mass balance check
for r in trial.reactions:
    print(r.id,  r.check_mass_balance())

HEX1 {'charge': -1.0}
PGI {}
FBP {'charge': 2.0}
PFK {'charge': -1.0}
FBA {}
TPI {}
GAPD {'charge': -3.0}
PGK {}
PGM {}
ENO {}
PEPCKm {}
PYK {'charge': 1.0}
PCm {'charge': 1.0}
LDH_L {'charge': -1.0}
PYRt2m {}
ADK1 {}
ATPM {'charge': 1.0}
NADHM {'charge': 1}
PEPtm {}
G6PP {'charge': 2}


In [11]:
trial.metabolites.h_c.charge= 1
trial.metabolites.h_m.charge= 1
# trial.metabolites.h_e.charge= 1
trial.metabolites.pi_c.charge= -2
trial.metabolites.pi_m.charge= -2

In [12]:
#Mass balance check
for r in trial.reactions:
    print(r.id,  r.check_mass_balance())

HEX1 {}
PGI {}
FBP {}
PFK {}
FBA {}
TPI {}
GAPD {}
PGK {}
PGM {}
ENO {}
PEPCKm {}
PYK {}
PCm {}
LDH_L {}
PYRt2m {}
ADK1 {}
ATPM {}
NADHM {'charge': 2}
PEPtm {}
G6PP {}


In [13]:
for met in ['glc__D_c', 
# 'pyr_c', 
'h2o_c', 'h_c', 'pi_c','lac__L_c','amp_c','o2_c','co2_c',
# 'atp_m',
'hco3_m', 'gtp_m',
    # 'h_m',
    #  'pi_m', 
    # 'adp_m',
     'gdp_m',
     'co2_m' 
     ]:
    rxn = cobra.Reaction('EX_%s' % met)
    trial.add_reaction(rxn)
    rxn.add_metabolites({met: -1})
    if met == 'glc__D_c':
        rxn.lower_bound = -1 # by convention negative exchange flux = uptake
    elif met == 'pyr_c':
        rxn.lower_bound = -1000
    else:
        rxn.lower_bound = -1000

In [14]:
# adding transport reactions to see if that changes anything
co2tm = {'co2_c': -1, 'co2_m': 1}
rxn = cobra.Reaction('CO2tm')
trial.add_reaction(rxn)
rxn.add_metabolites(co2tm)



atptm = {'adp_c': -1,'atp_m': -1, 'atp_c': 1,'adp_m': 1}
rxn = cobra.Reaction('ATPtm')
trial.add_reaction(rxn)
rxn.add_metabolites(atptm)


# H2Otm = {'h2o_c': -1, 'h2o_m': 1}
# rxn = cobra.Reaction('H2Otm')
# trial.add_reaction(rxn)
# rxn.add_metabolites(H2Otm)

# O2tm = {'o2_c': -1, 'o2_m': 1}
# rxn = cobra.Reaction('O2tm')
# trial.add_reaction(rxn)
# rxn.add_metabolites(O2tm)


htm = {'h_c': -1, 'h_m': 1}
rxn = cobra.Reaction('Htm')
trial.add_reaction(rxn)
rxn.add_metabolites(htm)



pit2m = {'h_c': -1,'pi_c': -1, 'h_m': 1,'pi_m': -1}
rxn = cobra.Reaction('PIt2m')
trial.add_reaction(rxn)
rxn.add_metabolites(pit2m)








In [ ]:
# for met in ['glc__D_c', 'pyr_c', 'h2o_c', 'h_c', 'pi_c','lac__L_c','amp_c','o2_c'
# 'atp_m','hco3_m', 'gtp_m','co2_c'
#     'h_m', 'pi_m', 'adp_m', 'gdp_m', 'co2_m' ]:
#     rxn = cobra.Reaction('EX_%s' % met)
#     trial.add_reaction(rxn)
#     rxn.add_metabolites({met: -1})
#     if met == 'glc__D_c':
#         rxn.lower_bound = -1 # by convention negative exchange flux = uptake
#     elif met == 'lac__L_c':
#         rxn.lower_bound = 0
#     else:
#         rxn.lower_bound = -1000

In [ ]:
# for reaction in trial.reactions:
#     print(reaction.id, reaction.lower_bound, reaction.upper_bound)

In [15]:
#View all the reactions that we need build
escher_builder = escher.Builder(
    model=trial,
    map_json=os.path.join(
        maps_dir,"Glycolysis_Gluconeogenesis.json"),highlight_missing=True)

escher_builder

Builder(highlight_missing=True)

In [25]:
trial.objective = 'ATPM'
flux_solution = trial.optimize()
flux_solution

,fluxes,reduced_costs
HEX1,1.0,0.0
PGI,1.0,0.0
FBP,0.0,0.0
PFK,1.0,0.0
FBA,1.0,0.0
...,...,...
EX_co2_m,0.0,-0.0
CO2tm,0.0,0.0
ATPtm,0.0,0.0
Htm,0.0,0.0


## trying to get minspan on perfect gluconeogenesis

In [17]:
reaction_list=[]
for reaction in trial.reactions:
    reaction_list.append(reaction.id)

reaction_list

['HEX1',
 'PGI',
 'FBP',
 'PFK',
 'FBA',
 'TPI',
 'GAPD',
 'PGK',
 'PGM',
 'ENO',
 'PEPCKm',
 'PYK',
 'PCm',
 'LDH_L',
 'PYRt2m',
 'ADK1',
 'ATPM',
 'NADHM',
 'PEPtm',
 'G6PP',
 'EX_glc__D_c',
 'EX_h2o_c',
 'EX_h_c',
 'EX_pi_c',
 'EX_lac__L_c',
 'EX_amp_c',
 'EX_o2_c',
 'EX_co2_c',
 'EX_hco3_m',
 'EX_gtp_m',
 'EX_gdp_m',
 'EX_co2_m',
 'CO2tm',
 'ATPtm',
 'Htm',
 'PIt2m']

In [20]:
# Initiate empty model
gluco= cobra.Model()
reactions_from_cobra=[
    # 'HEX1',
 'PGI',
 'FBP',
#  'PFK',
 'FBA',
 'TPI',
 'GAPD',
 'PGK',
 'PGM',
 'ENO',
 'PEPCKm',
#  'PYK',
 'PCm',
 'LDH_L',
 'PYRt2m',
 'ADK1',
 'ATPM',
 'NADHM',
 'PEPtm',
 'G6PP',
 'EX_glc__D_c',
 'EX_h2o_c',
 'EX_h_c',
 'EX_pi_c',
 'EX_lac__L_c',
 'EX_amp_c',
 'EX_o2_c',
 'EX_co2_c',
 'EX_hco3_m',
 'EX_gtp_m',
 'EX_gdp_m',
 'EX_co2_m',
 'CO2tm',
 'ATPtm',
 'Htm',
 'PIt2m']
for reaction in trial.reactions:
    if reaction.id in reactions_from_cobra:
        rxn=reaction
        gluco.add_reaction(rxn)

        # met=reaction.metabolites
        # temp=info['metabolites']
        # rxn.add_metabolites(met)

        print(rxn)



        # reaction_obj = cobra.Reaction(reaction)
        # reaction_obj.id=reaction
        # reaction= reaction_obj.id
        # reaction_obj.lower_bound = info['lower_bound']
        # reaction_obj.upper_bound = info['upper_bound']
        # reaction_obj.name = info['name']
        # trial.add_reaction(reaction_obj)
        # temp=info['metabolites']
        # reaction_obj.add_metabolites(temp)
        # print(reaction)
            


PGI: g6p_c <=> f6p_c
FBP: fdp_c + h2o_c --> f6p_c + pi_c
FBA: fdp_c <=> dhap_c + g3p_c
TPI: dhap_c <=> g3p_c
GAPD: g3p_c + nad_c + pi_c <=> 13dpg_c + h_c + nadh_c
PGK: 3pg_c + atp_c <=> 13dpg_c + adp_c
PGM: 2pg_c <=> 3pg_c
ENO: 2pg_c <=> h2o_c + pep_c
PEPCKm: gtp_m + oaa_m --> co2_m + gdp_m + pep_m
PCm: atp_m + hco3_m + pyr_m --> adp_m + h_m + oaa_m + pi_m
LDH_L: lac__L_c + nad_c <=> h_c + nadh_c + pyr_c
PYRt2m: h_c + pyr_c --> h_m + pyr_m
ADK1: amp_c + atp_c <=> 2.0 adp_c
ATPM: atp_c + h2o_c --> adp_c + h_c + pi_c
NADHM: nadh_c --> h_c + nad_c
PEPtm: pep_m + pi_c --> pep_c + pi_m
G6PP: g6p_c + h2o_c --> glc__D_c + pi_c
EX_glc__D_c: glc__D_c <=> 
EX_h2o_c: h2o_c <=> 
EX_h_c: h_c <=> 
EX_pi_c: pi_c <=> 
EX_lac__L_c: lac__L_c <=> 
EX_amp_c: amp_c <=> 
EX_o2_c: o2_c <=> 
EX_co2_c: co2_c <=> 
EX_hco3_m: hco3_m <=> 
EX_gtp_m: gtp_m <=> 
EX_gdp_m: gdp_m <=> 
EX_co2_m: co2_m <=> 
CO2tm: co2_c --> co2_m
ATPtm: adp_c + atp_m --> adp_m + atp_c
Htm: h_c --> h_m
PIt2m: h_c + pi_c + pi_m --> h_m


In [23]:
#View all the reactions that we need build
escher_builder = escher.Builder(
    model=gluco,
    map_json=os.path.join(
        maps_dir,"Glycolysis_Gluconeogenesis.json"),highlight_missing=True)

escher_builder

Builder(highlight_missing=True)

In [21]:
gluco.reactions.EX_glc__D_c.lower_bound=-1000

In [22]:
gluco.objective = 'EX_glc__D_c'
flux_solution = gluco.optimize()
flux_solution

,fluxes,reduced_costs
PGI,0.0,-0.00
FBP,0.0,0.00
FBA,0.0,-0.00
TPI,0.0,-0.00
GAPD,0.0,0.00
...,...,...
EX_co2_m,0.0,-0.00
CO2tm,0.0,0.00
ATPtm,0.0,0.00
Htm,0.0,-0.25


In [24]:
solved_fluxes = minspan(gluco, cores=3, verbose=False, timelimit=60)

df = pd.DataFrame(solved_fluxes.copy(), index=[i.id for i in gluco.reactions])
df = df/df.abs().max()
for col in range(len(df.columns)):
    column=df.iloc[:,col]
    vals = column.values
    min_val = min([abs(i) for i in vals if i != 0])
    corr_fac=1/min_val
    # print(corr_fac)
    vals=vals*abs(corr_fac)
    df['Norm'+ str(col)]=vals
csv_dir = os.path.join(minspan_dir,"gluco_csv.csv")
df.to_csv(csv_dir)

Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmp7t9bq0hm.lp
Reading time = 0.01 seconds
: 33 rows, 66 columns, 180 nonzeros
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmp7uxzs2tu.lp
Reading time = 0.01 seconds
: 99 rows, 99 columns, 366 nonzeros
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmpxznfom6z.lp
Reading time = 0.01 seconds
: 99 rows, 99 columns, 366 nonzeros


C:\Users\sicil\miniconda3\envs\massliver\lib\site-packages\cobra\util\solver.py:554 UserWarning: Solver status is 'infeasible'.


AttributeError: Unable to retrieve attribute 'X'

In [26]:
#Escher FBA: shows the solutions on the map
initial_flux_data = {
    reaction: flux
    for reaction, flux in flux_solution.fluxes.items()}

#view all the reactions that we need build
escher_builder = escher.Builder(
    model=trial,
    map_json=os.path.join(
        maps_dir,"Glycolysis_Gluconeogenesis.json")
    , reaction_data=initial_flux_data)

# Display map in notebook
escher_builder

Builder(reaction_data={'HEX1': 1.0, 'PGI': 1.0, 'FBP': 0.0, 'PFK': 1.0, 'FBA': 1.0, 'TPI': 1.0, 'GAPD': 2.0, '…

In [27]:
from cobra.io import json
import cobra.test
import os
from os.path import join
from glob import glob

In [28]:
gly_gluco_json = os.path.join(model_dir,"glycolysis_gluconeogenesis.json")
json.save_json_model(model=trial, filename=gly_gluco_json)

In [29]:
# import cplex 
# import gurobipy 

In [32]:
for model_file in glob(gly_gluco_json):
    model_name = model_file.split('/')[-1]
    if 'model' not in model_name:
        continue
    print(model_name)
    model= load_json_model(model_file)
    if 'NADPHM' in model.reactions:
        model.remove_reactions(['NADPHM'])
    # media = ['EX_lac__L_c', 'EX_pyr_c', 'EX_octa_c', 'EX_gln__L_c', 'EX_acetone_c', 'EX_bhb_c',
    #          'EX_glu__L_c', 'EX_ser__L_c', 'EX_cys__L_c', 'EX_gly_c', 'EX_ala__L_c', 'EX_so3_c',
    #         'EX_etoh_c', 'EX_fru_c']
    media = ['EX_glc__D_c']
    for met in media:
        if met in model.reactions:
            model.reactions.get_by_id(met).lower_bound = -1000.

    rxns = [i.id for i in model.reactions]
    # removed blocke reactions because it included ['PEPCKm', 'PCm', 'PYRt2m', 'ADK1', 'PEPtm', 'EX_h2o_c', 'EX_pi_c']
    # blocked = cobra.flux_analysis.find_blocked_reactions(model)
    # print(blocked)
    # model.remove_reactions(blocked)

    solved_fluxes = minspan(model, cores=3, verbose=False, timelimit=60)
    
    df = pd.DataFrame(solved_fluxes.copy(), index=[i.id for i in model.reactions])
    df = df/df.abs().max()
    for col in range(len(df.columns)):
        column=df.iloc[:,col]
        vals = column.values
        min_val = min([abs(i) for i in vals if i != 0])
        corr_fac=1/min_val
        # print(corr_fac)
        vals=vals*abs(corr_fac)
        df['Norm'+ str(col)]=vals
    csv_dir = os.path.join(minspan_dir,"gly_gluco_csv.csv")
    df.to_csv(csv_dir)
    # csv_minspan_dir = os.path.join(minspan_dir,"liver_csv_minspan.csv")
## WHere it's stored
csv_minspan_dir = os.path.join(minspan_dir,"gly_gluco_csv.csv")

c:\Users\sicil\LiverModel\models\glycolysis_gluconeogenesis.json
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmplst31pku.lp
Reading time = 0.01 seconds
: 33 rows, 72 columns, 210 nonzeros
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmpp1m7bud4.lp
Reading time = 0.01 seconds
: 105 rows, 108 columns, 411 nonzeros
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmppm43_34p.lp
Reading time = 0.01 seconds
: 105 rows, 108 columns, 411 nonzeros
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmpkkm5_ped.lp
Reading time = 0.01 seconds
: 105 rows, 108 columns, 411 nonzeros
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmpdnra5cay.lp
Reading time = 0.01 seconds
: 105 rows, 108 columns, 411 nonzeros
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmpj94rvzti.lp
Reading time = 0.01 seconds
: 105 rows, 108 columns, 411 nonzeros


AttributeError: Unable to retrieve attribute 'X'

In [ ]:
# making dataframe 
df = pd.read_csv(csv_minspan_dir,index_col=0) 
df

In [ ]:
import escher
from escher import Builder

In [ ]:
initial_minspan_data = {
    reaction: flux
    for reaction, flux in df.iloc[:,9].iteritems()}

In [ ]:
# #view all the reactions that we need build
# escher_builder = escher.Builder(
#     model=trial,
#     map_json=os.path.join(
#         maps_dir,"Glycolysis_Gluconeogenesis.json")
#     , reaction_data=initial_flux_data)


# New instance to prevent modifications to the existing maps
escher_builder = Builder(
    model=trial,
    map_json=os.path.join(
        maps_dir,"Glycolysis_Gluconeogenesis.json"),
    reaction_data=initial_minspan_data)

# Display map in notebook
escher_builder

In [ ]:
from cobra.io import json

In [ ]:
core_json = os.path.join(model_dir,"Glycolysis_Gluconeogenesis.json")
json.save_json_model(model=trial, filename=core_json)

In [ ]:
## Normalising minspan pathways
# for col in range(len(df.columns)):
#     column=df.iloc[:,col]
#     vals = column.values
#     min_val = min([abs(i) for i in vals if i != 0])
#     corr_fac=1/min_val
#     # print(corr_fac)
#     vals=vals*abs(corr_fac)
#     df['Norm'+ str(col)]=vals

### Making Df of metabolites and reactions, exporting them, and importing to form MASS model

In [ ]:
from cobra_dict import metabolite_to_dict as metabolite_to_dict
from cobra_dict import reaction_to_dict as reaction_to_dict

In [ ]:
met_df=pd.DataFrame()
for met in trial.metabolites:
    cobra_met= trial.metabolites.get_by_id(met.id)
    m=metabolite_to_dict(cobra_met)
    df_2=pd.DataFrame.from_dict(m,orient='index')
    df_2=df_2.T
    met_df=met_df.append(df_2)
met_df=met_df.set_index('id')
met_df

In [ ]:
csv_met_cobra = os.path.join(data_dir,"gly_gluco_cobra_met.csv")
met_df.to_csv(csv_met_cobra)

In [ ]:
rxn_df=pd.DataFrame()
for rxn in trial.reactions:
    cobra_rxn= trial.reactions.get_by_id(rxn.id)
    r=reaction_to_dict(cobra_rxn)
#     print(r)
    df=pd.DataFrame.from_dict(r,orient='index')
    df=df.T
    rxn_df=rxn_df.append(df)
rxn_df=rxn_df.set_index('id')
# rxn_df

In [ ]:
json_rxn_cobra = os.path.join(data_dir,"gly_gluco_cobra_rxn.json")
rxn_df.to_json(json_rxn_cobra,orient="index")

## Making MASS model of glycolysis

In [ ]:
liver= MassModel("Core_Model", array_type='DataFrame', dtype=np.int64)

#### Add the pathway metabolites and their information to a new MASS model from COBRA model

In [ ]:
# making dataframe of metabolites
import json
met_csv_df = pd.read_csv(csv_met_cobra,index_col="id")
mass_metabolite_info=met_csv_df.to_dict(orient='index')

# Loading reaction data as JSON file to maintain data types as dictionaries 

with open(json_rxn_cobra) as json_file:
    mass_rxn_data = json.load(json_file)

In [ ]:
#  Add all the remaining metabolites involved in the pathway
for met_id, info in mass_metabolite_info.items():
    met = MassMetabolite(met_id, name=info['name'], formula=info['formula'], 
                           charge=info['charge'], compartment=info['compartment'])
    liver.add_metabolites(met)

In [ ]:
for reaction, info in mass_rxn_data.items():
    reaction_obj = MassReaction(reaction)
    reaction_obj.id = reaction
    reaction= reaction_obj.id
    reaction_obj.lower_bound = info['lower_bound']
    reaction_obj.upper_bound = info['upper_bound']
    reaction_obj.name = info['name']
    liver.add_reactions(reaction_obj)
    temp=info['metabolites']
    reaction_obj.add_metabolites(temp)
    # print(reaction)

In [ ]:
liver.reactions.PGK.reverse_stoichiometry(inplace=True)
liver.reactions.PGM.reverse_stoichiometry(inplace=True)

## Setting initial conditions

In [ ]:
import csv
import openpyxl

In [ ]:
#Function to load data from the excel sheet
def load_data(filepath, sheet_name):
    """Load Liver data from an excel sheet"""
    df = pd.read_excel(engine='openpyxl',
        io=filepath,
        sheet_name=sheet_name,
        index_col=0)
    return df

In [ ]:
#Compare Results
excel_ic = os.path.join(data_dir,"Concentrations.xlsx")
# ic_df = pd.read_csv(csv_ic)
# ic_df.set_index()
ic_info_all = load_data(
    filepath=excel_ic,
    sheet_name="Concentrations")
ic_info_all.reset_index(level='ID', col_level=1, inplace=True)
ic_info_all

In [ ]:
conc_df =pd.DataFrame(ic_info_all.loc[:,["ID","Concentration (M)"]])
conc_df.set_index('ID',drop=True,inplace=True)
conc_df

In [ ]:
# ic_info= pd.DataFrame()
# ic_info.append

In [ ]:
print("Initial Conditions\n------------------")

for reaction in liver.reactions:
    #Setting inital condition of metabolties from HEPATOKIN  #M 
    for met in reaction.metabolites:
        if met.id in conc_df.index:
            mid=met.id
            metabolite = reaction.metabolites
            ic_value = conc_df.loc[mid,'Concentration (M)']
            row = [mid,ic_value]
            # column = [ic_value]
            ic_info.append(row)
            met.ic =  ic_value            
for metabolite, ic_value in liver.initial_conditions.items():
    print("{0}: {1}".format(metabolite, ic_value))

In [ ]:
ic_info

### Getting Standard and Physiological Gibbs energies of reactions to get Keq of reactions

In [ ]:
met_list = []
liver.boundary_conditions
for reaction in liver.boundary_conditions:
    print(reaction)

In [ ]:
# for reaction in liver.boundary:
#     excluded_reactions.append(reaction.id)
#     reaction.Keq=1    # Setting Keq
#     print(reaction,reaction.Keq)

In [ ]:
# Reactions that we don't want to get Keq from equilibrator [exchanges and psuedoreactions] SET THESE MANUALLY
# Setting Keq and kf for boundary reactions
print("Boundary Reactions and Values\n-----------------------------")
excluded_reactions=["ATPM", "NADHM","PYRt2m","PEPtm"]

for reaction in liver.boundary:
    excluded_reactions.append(reaction.id)
    reaction.Keq=1    # Setting Keq
    reaction.kf=1e6     # Setting kf
    #Setting boundary condition metabolties from ic of metabolite from HEPATOKIN   
    for met in reaction.reactants:
        if met.id in conc_df.index:
            mid=met.id
            boundary_met = reaction.boundary_metabolite
            bcvalue = conc_df.loc[mid,'Concentration (M)']
            liver.boundary_conditions[boundary_met] =  bcvalue
            bc_value = liver.boundary_conditions.get(boundary_met)
            print("{0}\n{1}: {2}\n".format(reaction, boundary_met, bc_value))

In [ ]:
1.32-0.08
1.24-0.08


In [ ]:
reaction_str= []
cc = ComponentContribution()
# optional: changing the aqueous environment parameters
cc.p_h = Q_(7.4)
cc.p_mg = Q_(3.31)
cc.ionic_strength = Q_("0.144M")
cc.temperature = Q_("310.15K")
R = 0.00831446261815324 #R = 0.00831446261815324 kJ/mol
from numpy import *
Keq_df=pd.DataFrame(columns=["Reaction","Keq"])
# Keq_df

In [ ]:
# Getting Keq from equilibrator by forming a reaction string that it can parse through
for reaction in liver.reactions:
    if reaction.id not in excluded_reactions:
        reactants = []
        #forming reaction string 
        for met in reaction.reactants:
            id_coeff= reaction.get_coefficient(met)
            new_id = met.id.rpartition("_" + str(met.compartment))[0] #removing compartment lettes, for eg "_c"
            if id_coeff == 1 or id_coeff == -1:
                new_id = "bigg.metabolite:" + new_id
            else:
                new_id = str(id_coeff) + " bigg.metabolite:" + new_id

            reactants.append(new_id)   
        reactants_string= " + ".join(reactants)
        reactants_string = str(reactants_string + " = ")

        # Forming product string
        products = []
        for met in reaction.products:
            new_id = met.id.rpartition("_" + str(met.compartment))[0] # removing compartment lettes, for eg "_c"
            new_id = "bigg.metabolite:" + new_id
            products.append(new_id)   
        products_string= " + ".join(products)

        #final reaction to parse
        reaction_to_parse = reactants_string + products_string         
        # print(reaction_to_parse)
        rxn_parsed = cc.parse_reaction_formula(reaction_to_parse)
        print(reaction.id)

        #Getting Gibbs energies
        dG0_prime = cc.standard_dg_prime(rxn_parsed)
    #         print(f"ΔG'° = {dG0_prime}")
        dGm_prime = cc.physiological_dg_prime(rxn_parsed)
        dG_prime_value_in_kj_per_mol = dGm_prime.value.m_as("kJ/mol")
        dG0prime_value_in_kj_per_mol = dG0_prime.value.m_as("kJ/mol")
    #     print(f"ΔG'm = {dG_prime_value_in_kj_per_mol}")
        delG = dG_prime_value_in_kj_per_mol

        # Calculating Keq from delG
        a = exp(-delG/ (R*310.15)) 
        reaction.Keq = a
        print(f"Keq = {a}")
#         list_keq=[reaction,a]
#         rxn_keq=pd.DataFrame(list_keq)
        df = pd.DataFrame([[reaction,a]], columns=["Reaction","Keq"])
        Keq_df.append(df)

# Keq_df    

In [ ]:
liver.reactions.NADHM.Keq=float("inf")
liver.reactions.ATPM.Keq=float("inf")
liver.reactions.PEPtm.Keq=1
liver.reactions.PYRt2m.Keq=1

In [ ]:
print("Equilibrium Constants\n---------------------")
for reaction in liver.reactions:
    print("{0}: {1}".format(reaction.Keq_str, reaction.Keq))

##  Set Fluxes

In [ ]:
flux_solution

### Getting minspan paths from file to form fluxes

#### Getting fluxes from FBA

In [ ]:
reaction_list=[]
for reaction in liver.reactions:
    reaction_list.append(reaction.id)


In [ ]:
for reaction, flux in flux_solution[reaction_list].iteritems():
    reaction = liver.reactions.get_by_id(reaction)
    reaction.steady_state_flux = flux 
#     * 1278.605 / 1000 # mmol/gdW*hr --> mmol/L * hr ---> mol/L*hr
    print("{0}: {1}".format(reaction.flux_symbol_str,
                            reaction.steady_state_flux))

# work on this function

In [ ]:
minspan_paths = [
    [1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 0, 0, 0, 2, 2, 0, 1, 0, 2, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 1, -1, 0, 1, 0, 0, 2, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]]

In [ ]:
def compute_steady_state_fluxes(
    self, pathways, independent_fluxes, update_reactions=False
):

In [ ]:
    def compute_steady_state_fluxes(
        self, pathways, independent_fluxes, update_reactions=False
    ):
        r"""Calculate the unique steady state flux for each reaction.

        The unique steady state flux for each reaction in the
        :class:`MassModel` is calculated using defined pathways, independently
        defined fluxes, and steady state concentrations, where index of values
        in the pathways must correspond to the index of the reaction in
        :attr:`MassModel.reactions`.

        Notes
        -----
        The number of individually defined fluxes must be the same as the
        number of pathways in order to determine the solution. For best
        results, the number of pathways to specify must equal the dimension
        of the right nullspace.

        Parameters
        ----------
        pathways : array-like
            An array-like object that define the pathways through the reaction
            network of the model. The given pathway vectors must be the same
            length as the number of reactions in the model, with indicies of
            values in the pathway vector corresponding to the indicies of
            reactions in the :attr:`reactions` attribute.
        independent_fluxes : dict
            A ``dict`` of steady state fluxes where :class:`~.MassReaction`\ s
            are keys and fluxes are values to utilize in order to calculate
            all other steady state fluxes. Must be the same length as the
            number of specified pathways.
        update_reactions : bool
            If ``True`` then update the :attr:`.MassReaction.steady_state_flux`
            with the calculated steady state flux value for each reaction.

        Return
        ------
        dict
            A ``dict`` where key:value pairs are the :class:`~.MassReaction`\ s
            with their corresponding calculated steady state fluxes.

        Warnings
        --------
        The indicies of the values in the pathway vector must correspond to the
        indicies of the reactions in the :attr:`reactions` attribute in order
        for the method to work as intended.

        """
        # Check inputs:
        if not isinstance(pathways, (np.ndarray, list)):
            raise TypeError(
                "Pathways must be numpy.ndarrays or array-like, "
                "such as a list of lists."
            )
        pathways = np.array(pathways)
        if len(self.reactions) != pathways.shape[1]:
            raise ValueError(
                "Pathways must have the same number of columns as"
                " the number of reactions in the model."
            )
        if not isinstance(independent_fluxes, dict):
            raise TypeError("independent_fluxes must be a dict")
        if not isinstance(update_reactions, bool):
            raise TypeError("update_reactions must be a bool")

        coeffs = []
        values = []
        # do get by ID
        for i, rxn in enumerate(self.reactions):
            if rxn in independent_fluxes:
                
                values.append(independent_fluxes[rxn])
                
                coeffs.append([path[i] for path in pathways])
        # Inverse coefficient matrix
        coeffs = np.linalg.inv(coeffs)

        # Obtain the inner product of values and coefficients,
        # then obtain the inner product of the pathways and first inner product
        flux_vector = np.inner(pathways.T, np.inner(coeffs, values))
        # Update the reactions if desired
        steady_state_fluxes = {}
        for i, rxn in enumerate(self.reactions):
            steady_state_flux = flux_vector[i]
            steady_state_fluxes.update({rxn: steady_state_flux})
            if update_reactions:
                rxn.steady_state_flux = steady_state_flux

        return steady_state_fluxes

In [ ]:
# initial_minspan_data = {
#     reaction: flux
#     for reaction, flux in df.iloc[:,5].iteritems()}

# df.iloc[:,5].values

In [ ]:
# minspan_paths = [
# list(df.iloc[:,5].values)]
# liver.compute_steady_state_fluxes(
#     pathways=minspan_paths,
#     independent_fluxes={
#         # SK_glc__D_c: 1.12,
#         # NADHM: .2 * 1.12,
#         # DM_amp_c: 0.014
#         },
#     update_reactions=True)

In [ ]:
# making dataframe of reactions

minspans = pd.read_csv(csv_minspan_dir,index_col=0) 

In [ ]:
minspans

## Set H2O/H Constants

In [ ]:
# Set concentrations of hydrogen, water as fixed
# Assume hydrogen in all compartments are fixed 
for metabolite in [
    "h_c",
#     'h_e',
'h_m',
#     'h2o_r','h2o_e',
    "h2o_c",
#     "glc__D_e", "pi_e",
'pi_m',
    'pi_c'
                  ]:
    metabolite = liver.metabolites.get_by_id(metabolite)
    metabolite.fixed = True

In [ ]:
for metabolite in [
    'h_m',
"h_c", 
#     'h_e',
"h2o_c",
#     'h2o_r','h2o_e' 
]:
    metabolite = liver.metabolites.get_by_id(metabolite)
    metabolite.ic = 1 #M

In [ ]:
#Formulate QP minimization list for concentrations
conc_solver = ConcSolver(
    liver, 
    excluded_metabolites=[
    "h_c",
#         'h_e',
 'h_m',
#     'h2o_r','h2o_e',
        "h2o_c",
#     "pi_e",
#     "glc__D_e", "lac__L_e",
#     "pyr_e"
    ], 
    excluded_reactions=[
        # "Ht", "H2Ot", "H2Oter",'PIter','PIt',
#     'G6Pter','GLCter','GLCt1',
'PYRt2m','PEPtm',
    # 'L_LACt2r','PYRt2'
    ],

    constraint_buffer=1)

conc_solver.setup_feasible_qp_problem(
fixed_conc_bounds=list(liver.fixed))
# fixed_Keq_bounds=liver.reactions.list_attr("Keq_str")
    
#assumption: provide uncertainity to allow for a variance for the Keqs rather than staying fixed
#conc_solver.setup_feasible_qp_problem(Keq_percent_deviation=0.1)

conc_solution = conc_solver.optimize()
conc_solution

In [ ]:
conc_solution.concentrations

In [ ]:
conc_df.index

In [ ]:
conc_comparison_fig, ax = plt.subplots(nrows=1, ncols=1,
                                       figsize=(5, 5))

plot_comparison(
    x=conc_df["Concentration (M)"], y=conc_solution,
    compare="concentrations",
    observable=[mid for mid in conc_df.index], ax=ax,
    legend="right outside", plot_function="loglog", 
    xlim=(1e-6,0.02), ylim=(1e-6,1e-1),
    xy_line=True,
    xy_legend="best", xlabel="Initial [mol/L]", ylabel="Adjusted [mol/L]")

conc_comparison_fig.tight_layout()
update_model_with_concentration_solution(
    liver, conc_solution, concentrations=True, inplace=True);

In [ ]:
# Keq_data = pd.read_csv("./data/analysis_data/equilibrium_constants.csv", index_col=0)

# Keq_data_comparison_fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5, 5))
# plot_comparison(
#     x=Keq_data["Initial"], y=Keq_data["Adjusted"], ax=ax,
#     title="Equilibrium Constants comparison",
#     legend="right outside", plot_function="loglog",
#     xlim=(1e-5, 1e5), ylim=(1e-5, 1e5), 
#     xy_line=True,
#     xy_legend="best", xlabel="Initial", ylabel="Adjusted")

# Keq_data_comparison_fig.tight_layout()

In [ ]:
Boundary= [
    "ATPM",
    
#     'EX_nad_c',
    # "EX_lac__L_c"
]
for x in Boundary:
    a=liver.reactions.get_by_id(x)
    a.kf=1e6

In [ ]:
 liver.steady_state_fluxes.items()

In [ ]:
liver.calculate_PERCs(fluxes={
                r: v for r, v in liver.steady_state_fluxes.items()
                if not (r.boundary  
#                         or r== liver.reactions.GLCter or r== glycogenolysis.reactions.PIter
#                                or r== glycogenolysis.reactions.G6Pter or r== glycogenolysis.reactions.PIt or 
#                                 r== glycogenolysis.reactions.PYRt2m or 
#                                 r== glycogenolysis.reactions.L_LACt2r or 
#                                 r== glycogenolysis.reactions.PYRt2 or 
#                                 r== glycogenolysis.reactions.PEPtm or r== glycogenolysis.reactions.GLCt1 
#                                 or r== glycogenolysis.reactions.Ht
                       )
},
                      update_reactions=True)
print(percs)                      

print("Forward Rate Constants\n----------------------")
for reaction in liver.reactions:
    print("{0}: {1:.6f}".format(reaction.kf_str, reaction.kf))

In [ ]:
#Function to add underscore in front of metabolite identifiers which start with a number
## fix this only after getting Keq from equlibrator because bigg doesn't have _13dpg for example
def prefix_number_id(id_str):
    """Prefix identifiers that start with numbers."""
    if re.match(r"^\d", id_str):
        id_str = "_" + id_str
    return id_str

In [ ]:
#Loop to edit the names using "prefix_number_id" function defined earlier
for metabolite in liver.metabolites:
    new_met_id = prefix_number_id(metabolite.id)
    metabolite.id = new_met_id
liver.repair()

In [ ]:
qcqa_model(liver, parameters=True, concentrations=True,
           fluxes=True, superfluous=True, elemental=True)

In [ ]:
# Setup simulation object
simG=Simulation(liver, verbose=True)
# Simulate from 0 to 1000 with 10001 points in the output
conc_sol, flux_sol = simG.simulate(liver, time=(0, 1e8))
# Quickly render and display time profiles
conc_sol.view_time_profile()

In [ ]:
#function to set x label and y label
def set_x_label(xlabel):
    if xlabel==None:
        xlabel="Time"
    else:
        xlabel=xlabel
    return xlabel
        
def set_y_label(ylabel):        
    if ylabel==None:
        ylabel="Concentrations"
    else:
        ylabel=ylabel
    return ylabel


In [ ]:
#function to make plot typelinear log
def set_plot_type(plot_type):
    if plot_type == None:
        x_plot_type = 'log'
        y_plot_type = 'log'
    elif plot_type == "logx":
        x_plot_type = 'log'
        y_plot_type = 'linear'
    elif plot_type == "logy":
        x_plot_type = 'linear'
        y_plot_type = 'log'
    elif plot_type == "linear":
        x_plot_type = 'linear'
        y_plot_type = 'linear'
    elif plot_type == "logxlogy":
        x_plot_type = 'log'
        y_plot_type = 'log'
    return x_plot_type,y_plot_type


In [ ]:
def view_all_conc_sol(conc_sol,
#                       dynamic=True
                      xlim=None,ylim=None,plot_type=None,xlabel=None,ylabel=None,width=None,height=None):
    """Generate an interactable time profile which can zoom into the plot with interactive tooltip"""
    #Shift click to click multiple
    
    data=conc_sol.to_frame()
    data = data.reset_index().melt('Time', var_name='Metabolites', value_name='Concentrations')
    
    # Setting X limit and Y limit
    def set_ylim(ylim):
        if ylim == None:
            conc=data['Concentrations']
            ylim_max=max(conc)
            ylim=[1e-5,ylim_max]
        else:
            ylim=ylim
        return ylim
    
    def set_xlim(xlim):
        if xlim == None:
            xlim_max=data['Time'].iloc[-1]
            xlim=[1e-5,xlim_max]

        else:
            xlim=xlim
        return xlim
    xlim=set_xlim(xlim)
    ylim=set_ylim(ylim)
        
       
    # Setting type of plot
    x_plot_type=set_plot_type(plot_type)[0]
    y_plot_type=set_plot_type(plot_type)[1]
        
    # Setting x label and y label
    xlabel=set_x_label(xlabel)
    ylabel=set_y_label(ylabel)
    # Setting width and height
    if width==None:
        width=500
    else:
        width=width
        
    if height==None:
        height=300
    else:
        height=height
    
    alt.data_transformers.disable_max_rows()
    selection = alt.selection_multi(fields=['Metabolites'])
    color = alt.condition(selection,
                          alt.Color('Metabolites:N', legend=None),
                          alt.value('lightgray'))

    line = alt.Chart(data).mark_line(clip=True).encode(alt.X('Time:Q',
                                                             title=xlabel,
                                                             scale=alt.Scale(type = x_plot_type,
                                                                             padding = 10,
                                                                             domain = xlim,
#                   zero=True,
                              ),
              axis=alt.Axis(tickCount=5)),
        alt.Y('Concentrations:Q',
              title=ylabel,
              scale=alt.Scale(type=y_plot_type,
                              padding = 10,
                              domain = ylim,
#                   zero=True, 
                             ),
               axis=alt.Axis(grid=False,
                             tickCount=5)),
        color=color,
        tooltip= ["Metabolites"]+["Time"]+["Concentrations"]
    ).properties(width=width, height=height).interactive()

    
    
#         if dynamic == False:
#             A = alt.layer(line)
#         else:
    legend = alt.Chart(data).mark_point().encode(
        y=alt.Y('Metabolites:N', axis=alt.Axis(orient='right')),
        color=color
    ).add_selection(
        selection
    )
    A= line | legend
    return A

In [ ]:
view_all_conc_sol(conc_sol,plot_type='logxlogy')